In [51]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-26 15:13:00--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2025-07-26 15:13:01 (13.6 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [52]:
# read dataset
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [53]:
len(text)

1115394

In [54]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [55]:
# details of dataset
chars = sorted(list(set(text))) # total unique characters
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [56]:
# Mapping sets
stoi = {ch:i for i ,ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
# encode str -> int
encode = lambda s:[stoi[c] for c in s]
# decode int -> str
decode = lambda l:''.join([itos[i] for i in l])

In [57]:
# encode dataset and store as tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape ,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [58]:
# split into train and val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [59]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [60]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"the context:{context} the target:{target}")

the context:tensor([18]) the target:47
the context:tensor([18, 47]) the target:56
the context:tensor([18, 47, 56]) the target:57
the context:tensor([18, 47, 56, 57]) the target:58
the context:tensor([18, 47, 56, 57, 58]) the target:1
the context:tensor([18, 47, 56, 57, 58,  1]) the target:15
the context:tensor([18, 47, 56, 57, 58,  1, 15]) the target:47
the context:tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target:58


In [61]:
train_data[76049], train_data[234249]

(tensor(24), tensor(44))

In [62]:

# creating the batch size
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,)) # predicting the 1st encoded index
  # print(f"the ix is :{ix}")
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("Outputs:")
print(yb.shape)
print(yb)

print('---------')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"the context:{context.tolist()} the target:{target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Outputs:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------
the context:[24] the target:43
the context:[24, 43] the target:58
the context:[24, 43, 58] the target:5
the context:[24, 43, 58, 5] the target:57
the context:[24, 43, 58, 5, 57] the target:1
the context:[24, 43, 58, 5, 57, 1] the target:46
the context:[24, 43, 58, 5, 57, 1, 46] the target:43
the context:[24, 43, 58, 5, 57, 1, 46, 43] the target:39
the context:[44] the target:53
the context:[44, 53] the target:56
the context:[44, 53, 56] the target:1
the context:[44, 53, 56, 1] the target:58
the context:[44, 53, 56, 1, 58] the target:46
the context:[44, 53, 56, 1, 58, 46] the target:39
the c

# Creating the biagram model

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BiagramModel(nn.Module):

  def __init__(self, vocab_size) -> None:
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, target= None):

    logits = self.token_embedding_table(idx)
    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)

      loss = F.cross_entropy(logits, target) # special case for multiple dimensions

    return logits , loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      # getting the predictions
      logits, loss = self(idx)
      # focus on last time step
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim = -1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim = 1)# (B, T+1)
    return idx

m = BiagramModel(vocab_size)
logits, loss  = m(xb, yb)
print(logits.shape, loss.shape)
print(loss)

print(decode(m.generate(idx= torch.zeros((1,1), dtype= torch.long), max_new_tokens= 100)[0].tolist()))

torch.Size([32, 65]) torch.Size([])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [64]:
# creating an optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [65]:
# Creating training model

batch_size = 32
for steps in range(1000):
  xb, yb = get_batch('train')

  # Evaluate loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none= True)
  loss.backward()
  optimizer.step()

print(loss.item())

3.7218432426452637


In [66]:
print(decode(m.generate(idx= torch.zeros((1,1), dtype= torch.long), max_new_tokens= 100)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-


# introducing the BOW(Bowl of words: where the words connects to their past words)

In [67]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [68]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    xbow[b, t] = torch.mean(xprev, 0)

In [69]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim= True)
xbow2 = wei @ x # (T, T) @ (B , T, C) -> (B,T,C)
# torch.isclose(xbow, xbow2)

In [70]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
# torch.isclose(xbow, xbow3)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


# Self-attention for small no of head

In [77]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single head
head_size = 16
query = nn.Linear(C, head_size, bias  = False)
key = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x)
q  = query(x)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v

In [78]:
out.shape

torch.Size([4, 8, 16])

# using the attention formula
` Attention(Q,K,V ) = softmax(QKT
 √
 dk
)V`

In [70]:
wei = q @ k.transpose(-2,-1) * head_size**-0.5